# Protein generator

### Import libraries

In [1]:
import urllib
import pandas as pd
from IPython.display import clear_output

### Function of retrieve conditional tags and sequences for a pfam_id from for uniprot

In [2]:
def get_sequence_and_conditional_tags(pfam_id):
    with urllib.request.urlopen(("https://www.uniprot.org/uniprot/?query=database%3A%28type%3Apfam+"+str(pfam_id)+"%29&precursor:no&columns=id,lineage(SUPERKINGDOM),lineage(KINGDOM),lineage(PHYLUM),lineage(CLASS),lineage(ORDER),linaege(FAMILY),lineage(GENUS),lineage(SPECIES),keywords,keyword-id,go(molecular%20function),go(cellular%20component),go(biological%20process),sequence&format=tab")) as f:
        query = f.read()
        query = str(query, 'utf-8')
    lines = query.split('\n')
    li_results = [x.split('\t') for x in lines]
    names = li_results[0]
    results = li_results[1:]
    return names, results

### Function to expand a column

In [3]:
def column_expander(df, key, splitkey):
    splits = df[key].str.split(splitkey, expand=True)
    new_columns = {}
    for split in range(len(splits.loc[0])):
        new_columns[split] = str(key)+"_"+str(split)
    splits.rename(columns = new_columns, inplace = True)
    df = pd.concat([df, splits], axis=1)
    return df 

### Download sequences, keywords and gene ontologies from uniprot

In [4]:
pfam_id = 'pf00123'
names, results = get_sequence_and_conditional_tags(pfam_id)

In [5]:
df = pd.DataFrame(results, columns=names)
df = df.dropna()

In [6]:
expand = {'Keyword ID':"; ",
          'Gene ontology (molecular function)':"; ",
          'Gene ontology (cellular component)':"; ",
          'Gene ontology (biological process)':"; ",
          'Keywords':";"}

for column in expand:
    df = column_expander(df, column, expand[column])
    df.drop(column, axis='columns', inplace=True)

In [7]:
seq = df['Sequence']

In [8]:
df.drop('Sequence', axis='columns', inplace=True)

In [9]:
df['Sequence'] = seq

In [11]:
df.to_csv("dataset.csv")

In [11]:
df.loc[1]

Entry                                                                          Q29W19
Taxonomic lineage (SUPERKINGDOM)                                            Eukaryota
Taxonomic lineage (KINGDOM)                                                   Metazoa
Taxonomic lineage (PHYLUM)                                                   Chordata
Taxonomic lineage (CLASS)                                                    Mammalia
                                                          ...                        
Keywords_10                                                                      None
Keywords_11                                                                      None
Keywords_12                                                                      None
Keywords_13                                                                      None
Sequence                            MTMCSGARLALLVYGILMHSSVYGSPAASGLRFPGIRPENEVYDED...
Name: 1, Length: 97, dtype: object